# 1. 生成对抗网络(GAN)

**生成模型**(Generative Model) 是一个广泛的机器学习领域，它是处理概率分布 $p(x)$ 的模型，其中 $x \in R_n$。这些模型是在某些潜在的高维空间中的数据点上定义的。通俗的理解，生成模型就是一类通过向真实数据分布学习，以此来确定其参数的模型，使得所学习的模型分布与真实数据分布尽可能地一致。但由于被建模随机变量的高维度，使得模型学习十分困难。

生成模型工作的流程：

1. 从一个已知分布（例如高斯分布或均匀分布）中随机采样样本；


2. 将该采样的样本输入到生成模型；


3. 生成模型根据真实分布反馈的信息不断更新自己的参数；


4. 经过若干次迭代，最后通过训练得到一个能够生成与真实数据尽可能一致的模型。

传统生成模型通常基于马尔可夫链、最大似然及近似推理，其代表模型有限制玻尔兹曼机(Restricted Boltzmann Machines, RBM)及其衍生模型如深度信念网络(Deep Belief Network， DBN)、深度波尔茨曼机(Deep Boltzmann Machines, DBM)、变分自动编码器(Variational Auto-Encoder, VAE)等，此类方法计算复杂且生成效果有限。

2014年， [Ian Goodfellow 等](https://arxiv.org/abs/1406.2661) 提出了基于博弈论的生成模型——**生成对抗网络**(Generative Adversarial Network, **GAN**)。该网络模型在生成图像数据方面的表现令人惊异，如今已经成为众多研究者的关注点。

除了图像合成，生成对抗网络在计算机视觉(CV)任务中还有很多的应用，如图像超分辨率、语义分割、图像编辑、图像修复、图像去噪、图像融合以及视频生成等。生成对抗网络在自然语言处理(NLP)中的应用也呈现日益增长的趋势， 例如：从文本生成图像、字体生成、对话生成、机器翻译、语音生成等。

# 2. GAN原理

## 2.1 GAN模型

受博弈论中二人零和博弈思想的启发，**GAN** 主要由**生成器**(Generator)和**判别器**(Discriminator)两个部分组成，**生成器**和**判别器**分别作为博弈的两方。其中，**生成器**的目的是生成接近真实的样本去骗过判别器， 而**判别器**是去区分真实的样本和生成的样本。通过对抗训练来不断的提高各自的能力，最终达到一个纳什均衡(Nash equilibrium)的状态。

- **生成器**$G$：它以一个随机向量（潜在空间中的一个随机点）作为输入，并将其解码为一张合成图像。


- **判别器**$D$：以一张图像（真实的或合成的均可）作为输入，并预测该图像是来自训练集还是由生成器创建。

![img](images/chapter17/GAN.jpg)

基于这样的思想可以设计出 GAN 的目标函数如下式所示：

$$ {\rm min} \; {\rm max}\; V(D, G) = \mathbb{E}_{x\sim p_ {\rm data}\;(x)}[{\rm log}D(x)] + \mathbb{E}_{z\sim p_ {\rm z}\;(z)}[{\rm log}(1-D(G(z)))] $$

其中 $G$ 代表生成器，$D$ 代表判别器，$P_{\rm data}(x)$ 代表真实样本的概率分布，$P_{z}(z)$ 代表随机噪声的概率分布，$z$ 是服从高斯分布的随机噪声。

> 对于生成器 $G$ 来说，通过随机噪声 $z$ 作为输入，生成器 $G$ 期望自己生成的样本尽可能地欺骗判别器 $D$，所以需要最大化判别概率 $D(G(z))$，于是对于生成器 $G$，它的目标函数是最小化 ${\rm log}(1-D(G(z)))$。

> 对于判别器 $D$，为了尽可能地区分真实样本和虚假的生成样本，它希望最小化判别概率 $D(G(z))$ 的同时，最大化判别概率 $D(x)$。于是判别器的目标函数是最大化 $ {\rm log} D(x) + {\rm log}(1-D(G(z)))$。

在对抗训练的过程中，判别器$D$ 判别真假样本的能力逐渐提高；而 生成器$G$ 为了欺骗判别器$D$，生成样本逐渐趋近于真实样本，最终使整个模型生成质量较好的新数据。

## 2.2 GAN的缺点

- 在模型训练过程中，存在模式崩溃问题：生成器从具有多种模式的训练集中仅学习到单个或有限的模式，而错失对其他模式的学习，从而无法生成具有多样性的样本。


- 使用随机向量作为生成器的输入缺乏语义和可操作性，无法控制模型生成具有指定特征的样本；


- 模型使用到的神经网络本身可以被进一步改造和优化；


- 训练过程中易出现梯度消失、训练不稳定、神经网络难以收敛等问题

# 3. 改进的GAN